In [57]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [58]:
data = pd.read_csv("C:/Users/hongz/Downloads/choices13k-main/reg_data_main2.csv")
# Modify graph_id as specified
data['graph_id'] = data['graph_id'] + 100 * data['reshuffle_ind']
data.head()

,subject_id,graph_id,time,click,risk,forecast,confidence,investment,reason_risky,reason_confidence,...,gender_ind,employment_ind,education_ind,income_ind,statistics_ind,risk-taking_ind,stock knowledge_ind,frequency _ind,history_ind,technical_ind
0,5fbb4426e47b46c3e2eeb544,156,11.550,6,6,112,62,59,NaN,NaN,...,0,0,1,1,0,0,0,0,0,0
1,65981b2c1df3be0020afa351,154,57.601,10,8,95,14,9,The ones that had big dips in them.,A lot of them. Only because I'm slowly trying ...,...,0,1,0,0,1,1,0,1,0,0
2,655791684bb1c5db02826d17,192,48.958,16,7,91,100,51,If it fluctuates more than 20% within 12 months.,With a $3 investment I do not feel unconfident...,...,0,1,1,1,0,1,0,1,1,0
3,62ddbd7eb3e9431e49b46ec1,182,49.950,11,8,115,79,60,NaN,NaN,...,1,1,1,0,1,1,0,0,0,0
4,5fb13091b87dfd5888f73e05,180,45.070,8,5,93,20,0,Those that have big drops.,When they are inconsistent and up and down on ...,...,0,0,0,0,0,1,0,1,1,0


## Transforming subject_id

In [59]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform 'subject_id' to numeric
data['subject_id_encoded'] = le.fit_transform(data['subject_id'])

data.head()

,subject_id,graph_id,time,click,risk,forecast,confidence,investment,reason_risky,reason_confidence,...,employment_ind,education_ind,income_ind,statistics_ind,risk-taking_ind,stock knowledge_ind,frequency _ind,history_ind,technical_ind,subject_id_encoded
0,5fbb4426e47b46c3e2eeb544,156,11.550,6,6,112,62,59,NaN,NaN,...,0,1,1,0,0,0,0,0,0,185
1,65981b2c1df3be0020afa351,154,57.601,10,8,95,14,9,The ones that had big dips in them.,A lot of them. Only because I'm slowly trying ...,...,1,0,0,1,1,0,1,0,0,461
2,655791684bb1c5db02826d17,192,48.958,16,7,91,100,51,If it fluctuates more than 20% within 12 months.,With a $3 investment I do not feel unconfident...,...,1,1,1,0,1,0,1,1,0,438
3,62ddbd7eb3e9431e49b46ec1,182,49.950,11,8,115,79,60,NaN,NaN,...,1,1,0,1,1,0,0,0,0,307
4,5fb13091b87dfd5888f73e05,180,45.070,8,5,93,20,0,Those that have big drops.,When they are inconsistent and up and down on ...,...,0,0,0,0,1,0,1,1,0,183


## Starting with regressions without FE

Baseline Regression

In [60]:
# Define independent variables for the baseline regression
X_base = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded']]

# Split the data
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, data['risk'], test_size=0.2, random_state=42)

# Baseline regression
model_base = LinearRegression()
model_base.fit(X_train_base, y_train)
y_pred_base = model_base.predict(X_test_base)
r2_base = r2_score(y_test, y_pred_base)
r2_base

0.03351046743638908

3-Feature Regression

In [61]:
X_3_feature = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor']]
X_train_3, X_test_3 = train_test_split(X_3_feature, test_size=0.2, random_state=42)

model_3_feature = LinearRegression()
model_3_feature.fit(X_train_3, y_train)
y_pred_3 = model_3_feature.predict(X_test_3)
r2_3_feature = r2_score(y_test, y_pred_3)
r2_3_feature

0.10873591492250123

8-Feature Regression

In [62]:
X_8_feature = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']]
X_train_8, X_test_8 = train_test_split(X_8_feature, test_size=0.2, random_state=42)

model_8_feature = LinearRegression()
model_8_feature.fit(X_train_8, y_train)
y_pred_8 = model_8_feature.predict(X_test_8)
r2_8_feature = r2_score(y_test, y_pred_8)
r2_8_feature

0.11383641839091119

8-Feature Lasso

In [16]:
model_8_lasso = LassoCV(cv=5)
model_8_lasso.fit(X_train_8, y_train)
y_pred_8_lasso = model_8_lasso.predict(X_test_8)
r2_8_lasso = r2_score(y_test, y_pred_8_lasso)
r2_8_lasso

0.11364455732747192

Large lasso

In [17]:
# Identify all 'ret_' and 'price_' columns
ret_columns = [col for col in data.columns if col.startswith('ret_')]
price_columns = [col for col in data.columns if col.startswith('price_')]

# Combine these columns with your other features
features = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis'] + ret_columns + price_columns
X_large_lasso = data[features]
X_train_large, X_test_large = train_test_split(X_large_lasso, test_size=0.2, random_state=42)

model_large_lasso = LassoCV(cv=5)
model_large_lasso.fit(X_train_large, y_train)
y_pred_large_lasso = model_large_lasso.predict(X_test_large)
r2_large_lasso = r2_score(y_test, y_pred_large_lasso)
r2_8_feature

0.11383641839091119

Huge Lasso

In [18]:
# Identify all 'ret_', 'price_', and their differences columns
ret_1st_diff_columns = [col for col in data.columns if '1st_diff' in col and col.startswith('ret_')]
ret_2nd_diff_columns = [col for col in data.columns if '2nd_diff' in col and col.startswith('ret_')]
price_1st_diff_columns = [col for col in data.columns if '1st_diff' in col and col.startswith('price_')]
price_2nd_diff_columns = [col for col in data.columns if '2nd_diff' in col and col.startswith('price_')]

# Combine these columns with your other features for the Huge Lasso model
features_huge_lasso = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis'] + ret_columns + price_columns + ret_1st_diff_columns + ret_2nd_diff_columns + price_1st_diff_columns + price_2nd_diff_columns
X_huge_lasso = data[features_huge_lasso]
X_train_huge, X_test_huge = train_test_split(X_huge_lasso, test_size=0.2, random_state=42)

model_huge_lasso = LassoCV(cv=5)
model_huge_lasso.fit(X_train_huge, y_train)
y_pred_huge_lasso = model_huge_lasso.predict(X_test_huge)
r2_huge_lasso = r2_score(y_test, y_pred_huge_lasso)
r2_huge_lasso

0.1453489107726199

Graph_id FE

In [19]:
graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')
data_with_fe = pd.concat([data, graph_id_dummies], axis=1)
#data_with_fe

In [21]:
# Assuming 'data' is your DataFrame and 'graph_id' is the categorical variable
graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')
data_with_fe = pd.concat([data, graph_id_dummies], axis=1)

# Create a list of features including the dummy variables
features = ['time', 'click', 'order', 'subject_id_encoded'] + list(graph_id_dummies.columns)

# Select these features for X
X_graph_id_fe = data_with_fe[features]

# Assuming y_train and y_test are defined or create them if not mentioned
X_train_graph_id_fe, X_test_graph_id_fe = train_test_split(X_graph_id_fe, test_size=0.2, random_state=42)

# Fit the baseline regression model
model_graph_id_fe = LinearRegression()
model_graph_id_fe.fit(X_train_graph_id_fe, y_train)

# Predict and calculate the R-squared value
y_pred_graph_id_fe = model_graph_id_fe.predict(X_test_graph_id_fe)
r2_graph_id_fe = r2_score(y_test, y_pred_graph_id_fe)

r2_graph_id_fe

0.169642524023483

## Starting with FE

Baseline Regression with subject_id_encoded Fixed Effects

In [22]:
# Create dummy variables for 'subject_id_encoded' and 'graph_id'
subject_id_dummies = pd.get_dummies(data['subject_id_encoded'], prefix='subject_id')
#graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')

# Add these dummy variables to the data
data_with_dummies = pd.concat([data, subject_id_dummies], axis=1)
data_with_dummies = pd.concat([data_with_dummies, graph_id_dummies], axis=1)
# Define the baseline features including the subject_id dummies
baseline_features_with_dummies = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order'] + list(subject_id_dummies.columns)
#baseline_features_with_dummies

In [23]:
# Define the independent and dependent variables
X_baseline_with_dummies = data_with_dummies[baseline_features_with_dummies]
y = data_with_dummies['risk']

# Split the data into training and testing sets
X_train_baseline, X_test_baseline, y_train, y_test = train_test_split(X_baseline_with_dummies, y, test_size=0.2, random_state=42)

# Fit the baseline regression model
model_baseline_with_dummies = LinearRegression()
model_baseline_with_dummies.fit(X_train_baseline, y_train)

# Predict and calculate the R-squared value
y_pred_baseline = model_baseline_with_dummies.predict(X_test_baseline)
r2_baseline_fe = r2_score(y_test, y_pred_baseline)

r2_baseline_fe

0.160245352228657

3-Feature Regression with subject_id_encoded Fixed Effects

In [24]:
# Define 3-feature model variables
features_3_feature = baseline_features_with_dummies + ['Recency_Factor', 'Rep_Factor', 'Sign_Factor']

# Split the data for 3-feature model
X_3_feature = data_with_dummies[features_3_feature]
X_train_3_feature, X_test_3_feature, y_train, y_test = train_test_split(X_3_feature, y, test_size=0.2, random_state=42)

# Fit the 3-feature model
model_3_feature = LinearRegression()
model_3_feature.fit(X_train_3_feature, y_train)

# Predict and calculate R-squared
y_pred_3_feature = model_3_feature.predict(X_test_3_feature)
r2_3_feature_fe = r2_score(y_test, y_pred_3_feature)
r2_3_feature_fe

0.2583782289217412

8-Feature Regression with subject_id_encoded Fixed Effects

In [25]:
# Define 8-feature model variables including 'subject_id_encoded' dummies
features_8_feature = baseline_features_with_dummies + ['Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']

# Split the data for 8-feature model
X_8_feature = data_with_dummies[features_8_feature]
X_train_8_feature, X_test_8_feature, y_train, y_test = train_test_split(X_8_feature, y, test_size=0.2, random_state=42)

# Fit the 8-feature model
model_8_feature = LinearRegression()
model_8_feature.fit(X_train_8_feature, y_train)

# Predict and calculate R-squared
y_pred_8_feature = model_8_feature.predict(X_test_8_feature)
r2_8_feature_fe = r2_score(y_test, y_pred_8_feature)
r2_8_feature_fe

0.2628515051965741

8 Features Lasso with subject_id_encoded Fixed Effects

In [26]:
# Fit the Large Lasso model
model_8_feature_lasso_fe = LassoCV(cv=5)
model_8_feature_lasso_fe.fit(X_train_8_feature, y_train)

# Predict and calculate R-squared
y_pred_8_feature_lasso_fe = model_8_feature_lasso_fe.predict(X_test_8_feature)
r2_8_feature_lasso_fe = r2_score(y_test, y_pred_8_feature_lasso_fe)
r2_8_feature_lasso_fe

0.2153121124370303

Large Lasso with subject_id_encoded Fixed Effects

In [27]:
# Define Large Lasso model variables including additional features and 'subject_id_encoded' dummies
features_large_lasso_fe = features_8_feature + ret_columns + price_columns  # Assuming ret_columns and price_columns are defined

# Split the data for Large Lasso model
X_large_lasso_fe = data_with_dummies[features_large_lasso_fe]
X_train_large_lasso_fe, X_test_large_lasso_fe, y_train, y_test = train_test_split(X_large_lasso_fe, y, test_size=0.2, random_state=42)

# Fit the Large Lasso model
model_large_lasso_fe = LassoCV(cv=5)
model_large_lasso_fe.fit(X_train_large_lasso_fe, y_train)

# Predict and calculate R-squared
y_pred_large_lasso_fe = model_large_lasso_fe.predict(X_test_large_lasso_fe)
r2_large_lasso_fe = r2_score(y_test, y_pred_large_lasso_fe)
r2_large_lasso_fe

0.15328066023148346

Huge Lasso with subject_id_encoded Fixed Effects

In [28]:
# Assuming 'data_with_dummies' is correctly prepared and includes all required dummy variables
features_huge_lasso_fe = features_large_lasso_fe + ret_1st_diff_columns + ret_2nd_diff_columns + price_1st_diff_columns + price_2nd_diff_columns  # ensure these are correctly defined

# Select features
X_huge_lasso_fe = data_with_dummies[features_huge_lasso_fe]
y = data_with_dummies['risk']  # make sure 'risk' is the correct target column

# Split the data
X_train_huge_lasso_fe, X_test_huge_lasso_fe, y_train, y_test = train_test_split(X_huge_lasso_fe, y, test_size=0.2, random_state=42)

# Fit the Huge Lasso model
model_huge_lasso_fe = LassoCV(cv=5)
model_huge_lasso_fe.fit(X_train_huge_lasso_fe, y_train)

# Predict and calculate R-squared
y_pred_huge_lasso_fe = model_huge_lasso_fe.predict(X_test_huge_lasso_fe)
r2_huge_lasso_fe = r2_score(y_test, y_pred_huge_lasso_fe)

print(f"R-squared for Huge Lasso with fixed effects: {r2_huge_lasso_fe:.4f}")

R-squared for Huge Lasso with fixed effects: 0.1532


Graph FE Regression

In [55]:
graph_fe_features = ['time', 'click', 'order', 'subject_id_encoded'] + list(subject_id_dummies.columns) + list(graph_id_dummies.columns)

In [56]:
# Define the independent and dependent variables
X_graph_with_dummies = data_with_dummies[graph_fe_features]
y = data_with_dummies['risk']

# Split the data into training and testing sets
X_train_graph_id, X_test_graph_id, y_train, y_test = train_test_split(X_graph_with_dummies, y, test_size=0.2, random_state=42)

# Fit the baseline regression model
model_graph_id_with_dummies = LinearRegression()
model_graph_id_with_dummies.fit(X_train_graph_id, y_train)

# Predict and calculate the R-squared value
y_pred_graph_id = model_graph_id_with_dummies.predict(X_test_graph_id)
r2_graph_fe = r2_score(y_test, y_pred_graph_id)
r2_graph_fe

KeyError: "['graph_id_-113.55', 'graph_id_-111.0', 'graph_id_-109.8', 'graph_id_-109.1', 'graph_id_-108.75', 'graph_id_-108.15', 'graph_id_-107.8', 'graph_id_-107.65', 'graph_id_-106.65', 'graph_id_-106.55', 'graph_id_-105.9', 'graph_id_-105.85', 'graph_id_-105.5', 'graph_id_-105.05', 'graph_id_-105.0', 'graph_id_-104.75', 'graph_id_-104.25', 'graph_id_-104.2', 'graph_id_-103.9', 'graph_id_-103.55', 'graph_id_-103.3', 'graph_id_-103.15', 'graph_id_-103.1', 'graph_id_-103.05', 'graph_id_-103.0', 'graph_id_-102.95', 'graph_id_-102.9', 'graph_id_-102.85', 'graph_id_-102.8', 'graph_id_-102.7', 'graph_id_-102.65', 'graph_id_-102.4', 'graph_id_-102.3', 'graph_id_-102.1', 'graph_id_-102.0', 'graph_id_-101.9', 'graph_id_-101.8', 'graph_id_-101.75', 'graph_id_-101.7', 'graph_id_-101.6', 'graph_id_-101.45', 'graph_id_-101.4', 'graph_id_-101.3', 'graph_id_-101.2', 'graph_id_-101.1', 'graph_id_-100.9', 'graph_id_-100.85', 'graph_id_-100.65', 'graph_id_-100.6', 'graph_id_-100.55', 'graph_id_-100.45

Leave-One-Out

In [31]:
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

def compute_lasso_loo(data, features, target):
    subject_ids = data['subject_id_encoded'].unique()
    overall_scores = []

    for _ in tqdm(range(2), desc="Overall LOO Repeats"):  # tqdm for visual progress on the 10 repetitions
        scores = []
        for subject_id in tqdm(subject_ids, desc="Processing Subject IDs", leave=False):  # Nested tqdm for subjects
            subject_data = data[data['subject_id_encoded'] == subject_id]
            X = subject_data[features]
            y = subject_data[target]
            if X.empty:
                print(f"Warning: No data for subject_id {subject_id}. Skipping.")
                continue
            
            # Additional diagnostics
            if X.isnull().any().any() or y.isnull().any():
                print(f"NaN values found in features or target for subject_id {subject_id}")
                continue
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
            
            if len(X_train) < 2:
                print(f"Not enough samples to train for subject_id {subject_id}")
                continue

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            lasso_cv = LassoCV(cv=max(2, int(len(X_train) * 0.9)), random_state=42)
            lasso_cv.fit(X_train_scaled, y_train)
            
            if lasso_cv.alpha_ == 0:
                print(f"Alpha value of zero encountered for subject_id {subject_id}. This can lead to a degenerate model.")

            lasso = Lasso(alpha=lasso_cv.alpha_)
            lasso.fit(X_train_scaled, y_train)

            loo_scores = cross_val_score(lasso, X_test_scaled, y_test, cv=LeaveOneOut(), scoring='r2')
            if np.isnan(np.mean(loo_scores)):
                print(f"NaN R-squared for subject_id {subject_id}. Check data quality and model configuration.")
            scores.append(np.mean(loo_scores))
        
        overall_scores.append(np.mean(scores))
    
    mean_score = np.mean(overall_scores)
    return mean_score

# Example usage with diagnostic outputs
features_base = ['graph_id', 'time', 'click', 'forecast', 'investment', 'gender_ind', 'employment_ind', 'education_ind', 'income_ind']
mean_score_lasso_8 = compute_lasso_loo(data, features_base, 'risk')
print("8-feature Lasso Mean Score:", mean_score_lasso_8)


Processing Subject IDs:   0%|                                                          | 1/502 [00:00<05:59,  1.39it/s]

NaN R-squared for subject_id 185. Check data quality and model configuration.



Processing Subject IDs:   0%|▏                                                         | 2/502 [00:01<04:54,  1.70it/s]

NaN R-squared for subject_id 461. Check data quality and model configuration.



Processing Subject IDs:   1%|▎                                                         | 3/502 [00:01<04:47,  1.74it/s]

...

NaN R-squared for subject_id 443. Check data quality and model configuration.



Processing Subject IDs: 100%|███████████████████████████████████████████████████████▉| 501/502 [04:28<00:00,  1.98it/s]

NaN R-squared for subject_id 294. Check data quality and model configuration.



Overall LOO Repeats: 100%|██████████████████████████████████████████████████████████████| 2/2 [09:22<00:00, 281.27s/it]

NaN R-squared for subject_id 164. Check data quality and model configuration.
8-feature Lasso Mean Score: nan


## Lasso

In [32]:
def lasso_kfold_cv(data, features, target):
    X = data[features]
    y = data[target]

    # Standardizing the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Setting up 10-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Using LassoCV to find the best alpha during cross-validation
    lasso_cv = LassoCV(alphas=np.logspace(-6, 0, 100), cv=kf, random_state=42)
    lasso_cv.fit(X_scaled, y)

    # Evaluating the model using cross-validation
    cv_scores = cross_val_score(lasso_cv, X_scaled, y, cv=kf, scoring='r2')
    mean_r2 = np.mean(cv_scores)
    std_r2 = np.std(cv_scores)

    return mean_r2, std_r2

# Using the model
features_base = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']
r2_8_feature_lasso, std_8_feature_lasso = lasso_kfold_cv(data, features_base, 'risk')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso:.4f} ± {std_8_feature_lasso:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.1180 ± 0.0092


In [33]:
# Extracting ret and price features
ret_features = [col for col in data.columns if col.startswith('ret_') and 'diff' not in col]
price_features = [col for col in data.columns if col.startswith('price_') and 'diff' not in col]

# Large Lasso features include base features plus all ret_ and price_ values
features_large = features_base + ret_features + price_features

# Adding first and second differences for Huge Lasso features
ret_diff_features = [col for col in data.columns if 'ret_' in col and 'diff' in col]
price_diff_features = [col for col in data.columns if 'price_' in col and 'diff' in col]

features_huge = features_large + ret_diff_features + price_diff_features

# Calculate R-squared for Large Lasso
r2_large_lasso, std_r2_large = lasso_kfold_cv(data, features_large, 'risk')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso:.4f} ± {std_r2_large:.4f}")

# Calculate R-squared for Huge Lasso
r2_huge_lasso, std_r2_huge = lasso_kfold_cv(data, features_huge, 'risk')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso:.4f} ± {std_r2_huge:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.1525 ± 0.0085
Mean R-squared for Huge Lasso with 10-Fold CV: 0.1522 ± 0.0083


lasso within subjected_id

In [34]:
from sklearn import set_config
set_config(enable_metadata_routing=False)  # Explicitly disable metadata routing

def lasso_group_kfold_cv(data, features, target):
    X = data[features]
    y = data[target]
    groups = data['subject_id_encoded']  # This assumes subject_id_encoded is a column in the DataFrame

    # Setting up 10-fold cross-validation that respects groups
    gkf = GroupKFold(n_splits=10)

    # Create a pipeline to scale data and apply Lasso
    lasso_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso_cv', LassoCV(alphas=np.logspace(-6, 0, 100), max_iter=10000, random_state=42))
    ])

    # Compute cross-validated R^2 scores
    cv_scores = cross_val_score(lasso_pipeline, X, y, groups=groups, cv=gkf, scoring='r2')
    mean_r2 = np.mean(cv_scores)
    std_r2 = np.std(cv_scores)

    return mean_r2, std_r2

# Usage example
features_base = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']
r2_8_feature_lasso, std_8_feature_lasso = lasso_group_kfold_cv(data, features_base, 'risk')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso:.4f} ± {std_8_feature_lasso:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.1155 ± 0.0463


In [35]:
# Large Lasso
r2_large_lasso, std_r2_large_lasso = lasso_group_kfold_cv(data, features_large, 'risk')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso:.4f} ± {std_r2_large_lasso:.4f}")

# Huge Lasso
r2_huge_lasso, std_r2_huge_lasso = lasso_group_kfold_cv(data, features_huge, 'risk')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso:.4f} ± {std_r2_huge_lasso:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.1507 ± 0.0519
Mean R-squared for Huge Lasso with 10-Fold CV: 0.1509 ± 0.0522


With control of subject_id_encoded FE

In [36]:
r2_8_feature_lasso_fe, std_8_feature_lasso = lasso_kfold_cv(data_with_dummies, features_base, 'risk')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso_fe:.4f} ± {std_8_feature_lasso:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.1180 ± 0.0092


In [37]:
r2_large_lasso_fe, std_r2_large_lasso = lasso_group_kfold_cv(data_with_dummies, features_large, 'risk')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso_fe:.4f} ± {std_r2_large_lasso:.4f}")

# Huge Lasso
r2_huge_lasso_fe, std_r2_huge_lasso = lasso_group_kfold_cv(data_with_dummies, features_huge, 'risk')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso_fe:.4f} ± {std_r2_huge_lasso:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.1507 ± 0.0519
Mean R-squared for Huge Lasso with 10-Fold CV: 0.1509 ± 0.0522


In [54]:
# Creating the DataFrame
dak = {
    "Model((Risk))": [
        "Baseline Regression", "3-feature Regression", "8-feature Regression",
        "8-feature Lasso", "Large Lasso", "Huge Lasso", "Graph FE Regression"
    ],
    "R-squared (Without Subject ID FE)": [
        r2_base, r2_3_feature, r2_8_feature,
        r2_8_feature_lasso, r2_large_lasso, r2_large_lasso, r2_graph_id_fe
    ],
    "R-squared (With Subject ID FE)": [
        r2_baseline_fe, r2_3_feature_fe, r2_8_feature_fe,
        r2_8_feature_lasso_fe, r2_large_lasso_fe, r2_large_lasso_fe, r2_graph_fe
    ],
    "R-squared (Demeaned)": [
        r2_base_demeaned, r2_3_feature_demeaned, r2_8_feature_demeaned,
        r2_8_feature_lasso_demeaned, r2_large_lasso_demeaned, r2_large_lasso_demeaned, r2_graph_id_fe_demeaned
    ]
}

df = pd.DataFrame(dak)

df

,Model((Risk)),R-squared (Without Subject ID FE),R-squared (With Subject ID FE),R-squared (Demeaned)
0,Baseline Regression,0.033510,0.160245,1.064144e-02
1,3-feature Regression,0.108736,0.258378,1.762215e-02
2,8-feature Regression,0.113836,0.262852,1.477908e-02
3,8-feature Lasso,0.115514,0.117984,1.290238e-02
4,Large Lasso,0.150726,0.150726,1.285531e-02
5,Huge Lasso,0.150726,0.150726,1.285531e-02
6,Graph FE Regression,0.169643,0.310335,-3.257372e+20


In [40]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


# Demean all variables except 'subject_id'
grouped = data.groupby('subject_id')
data_demeaned = grouped.transform(lambda x: x - x.mean())

# 将 'risk' 也进行去均值化
#data_demeaned['risk'] = data['risk'] - grouped['risk'].transform('mean')


In [48]:
# Define independent variables for the baseline regression
X_base = data_demeaned[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded']]
y = data_demeaned['risk']
# Split the data
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, data_demeaned['confidence'], test_size=0.2, random_state=42)

# Baseline regression
model_base_demeaned = LinearRegression()
model_base_demeaned.fit(X_train_base, y_train)
y_pred_base_demeaned = model_base_demeaned.predict(X_test_base)
r2_base_demeaned = r2_score(y_test, y_pred_base_demeaned)
r2_base_demeaned

0.01064144361972097

In [49]:
X_3_feature = data_demeaned[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor']]
X_train_3, X_test_3 = train_test_split(X_3_feature, test_size=0.2, random_state=42)

model_3_feature_demeaned = LinearRegression()
model_3_feature_demeaned.fit(X_train_3, y_train)
y_pred_3_demeaned = model_3_feature_demeaned.predict(X_test_3)
r2_3_feature_demeaned = r2_score(y_test, y_pred_3_demeaned)
r2_3_feature_demeaned

0.017622149443554003

In [50]:
X_8_feature = data_demeaned[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']]
X_train_8, X_test_8 = train_test_split(X_8_feature, test_size=0.2, random_state=42)

model_8_feature = LinearRegression()
model_8_feature.fit(X_train_8, y_train)
y_pred_8 = model_8_feature.predict(X_test_8)
r2_8_feature_demeaned = r2_score(y_test, y_pred_8)
r2_8_feature_demeaned

0.014779082636650509

In [51]:
r2_8_feature_lasso_demeaned, std_8_feature_lasso_demeaned = lasso_kfold_cv(data_demeaned, features_base, 'confidence')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso_demeaned:.4f} ± {std_8_feature_lasso_demeaned:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.0129 ± 0.0059


In [52]:
# Calculate R-squared for Large Lasso
r2_large_lasso_demeaned, std_r2_large_demeaned = lasso_kfold_cv(data_demeaned, features_large, 'confidence')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso_demeaned:.4f} ± {std_r2_large_demeaned:.4f}")

# Calculate R-squared for Huge Lasso
r2_huge_lasso_demeaned, std_r2_huge_demeaned = lasso_kfold_cv(data_demeaned, features_huge, 'confidence')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso_demeaned:.4f} ± {std_r2_huge_demeaned:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.0129 ± 0.0081
Mean R-squared for Huge Lasso with 10-Fold CV: 0.0118 ± 0.0079


In [53]:
# Assuming 'data' is your DataFrame and 'graph_id' is the categorical variable
graph_id_dummies = pd.get_dummies(data_demeaned['graph_id'], prefix='graph_id')
data_with_fe = pd.concat([data_demeaned, graph_id_dummies], axis=1)

# Create a list of features including the dummy variables
features = ['time', 'click', 'order', 'subject_id_encoded'] + list(graph_id_dummies.columns)

# Select these features for X
X_graph_id_fe = data_with_fe[features]

# Assuming y_train and y_test are defined or create them if not mentioned
X_train_graph_id_fe, X_test_graph_id_fe = train_test_split(X_graph_id_fe, test_size=0.2, random_state=42)
#X_train_graph_id_fe, X_test_graph_id_fe, y_train, y_test = train_test_split(X_graph_id_fe, data['confidence'], test_size=0.2, random_state=42)

# Fit the baseline regression model
model_graph_id_fe = LinearRegression()
model_graph_id_fe.fit(X_train_graph_id_fe, y_train)

# Predict and calculate the R-squared value
y_pred_graph_id_fe = model_graph_id_fe.predict(X_test_graph_id_fe)
r2_graph_id_fe_demeaned = r2_score(y_test, y_pred_graph_id_fe)

r2_graph_id_fe_demeaned

-3.257371509422882e+20